In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
df=pd.read_excel('clean_data.xlsx')
df.head()

,Date,Time,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [3]:
seasons={}

In [4]:
spring = df.loc[df['Date']<'2004-06-01']
spring = spring.reset_index(drop=True)
seasons['spring']=spring

In [5]:
summer = df.loc[(df['Date']<'2004-09-01') & (df['Date']>='2004-06-01')]
summer = summer.reset_index(drop=True)
seasons['summer']=summer

In [6]:
autumn = df.loc[(df['Date']<'2004-12-01') & (df['Date']>='2004-09-01')]
autumn = autumn.reset_index(drop=True)
seasons['autumn']=autumn

In [7]:
winter = df.loc[(df['Date']<'2005-03-01') & (df['Date']>='2004-12-01')]
winter = winter.reset_index(drop=True)
seasons['winter']=winter

In [8]:
summer.to_excel('summer.xlsx', index=False)
spring.to_excel('spring.xlsx', index=False)
autumn.to_excel('autumn.xlsx', index=False)
winter.to_excel('winter.xlsx', index=False)

In [60]:
# plt.scatter(X, y)
# y_pred = reg.predict(X)
# plt.plot(X, y_pred, color ='k')
# plt.xlabel("X axis label")
# plt.ylabel("Y axis label")
# plt.show()

In [61]:
# reg.coef_

In [62]:
# reg.intercept_

In [8]:
cols = df.columns
cols

Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)',
       'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
       'T', 'RH', 'AH'],
      dtype='object')

In [41]:
for season in seasons:
    for col in cols[2:-3]:
        X=seasons[season]['T'].values.reshape(-1,1)
        y=seasons[season][col].values.reshape(-1,1)

        reg = LinearRegression().fit(X, y)
        r2 = reg.score(X, y)
        c = reg.coef_
        i = reg.intercept_

        plt.scatter(X, y)
        y_pred = reg.predict(X)

        plt.plot(X, y_pred, color ='k')
        plt.xlabel('Temperature')
        plt.ylabel(col)
        plt.suptitle(season)
        plt.title(('coef = ',np.round(c,2)[0][0],' intercept = ',np.round(i,2)[0]))
#         plt.show()
        plt.savefig(season+' '+col+'.png')
        plt.close()

In [13]:
summer.corr().to_excel('summer_corr.xlsx')
winter.corr().to_excel('winter_corr.xlsx')
autumn.corr().to_excel('autumn_corr.xlsx')
spring.corr().to_excel('spring_corr.xlsx')

In [97]:
y=spring['T'].values.reshape(-1,1)
x=[]
for col in spring.columns:
    if col not in ['Date', 'Time', 'T']:
        x.append(spring[col].values)
x=np.transpose(x)
# x=np.reshape(x,(len(x), len(x[0])))

In [121]:
from sklearn.metrics import mean_absolute_error

In [98]:
reg = LinearRegression().fit(x, y)

In [123]:
reg.score(x,y)

0.9452115562551225

In [124]:
y_pred=reg.predict(x)
mean_absolute_error(y, y_pred)

0.9478051504914674

In [74]:
test = pd.read_excel('clean_data.xlsx')

In [75]:
test = test[(test['Date']>='2005-03-01') & (test['Date']<='2005-03-31')]

In [76]:
test=test.reset_index(drop=True)

In [119]:
y_test=test['T'].values.reshape(-1,1)
x_test=[]
for col in test.columns:
    if col not in ['Date', 'Time', 'T']:
        x_test.append(test[col].values)
x_test=np.transpose(x_test)
x_test=np.reshape(x_test,(len(x_test), len(x_test[0])))

In [78]:
reg.score(x_test,y_test)

0.6920548918190437

In [79]:
from keras.layers import Flatten, Dense, BatchNormalization
from keras import Input, layers
from keras.models import Model

In [117]:
n_units=50
input_tensor = Input(shape=(len(x[0]),))
dense = Dense(n_units, activation='relu')(input_tensor)

for i in range(5):
    dense = Dense(n_units, activation='relu')(dense)
    
dense=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(dense)
output_tensor = Dense(1, activation='relu')(dense)

model = Model(input_tensor, output_tensor)

model.compile(optimizer='sgd', loss='mse')

history = model.fit(x, y, epochs=1000, batch_size=32)

Epoch 1/1000
1974/1974 [==============================] - 1s 507us/step - loss: 66.0016
Epoch 2/1000
1974/1974 [==============================] - 0s 201us/step - loss: 23.0973
Epoch 3/1000
1974/1974 [==============================] - 0s 214us/step - loss: 19.8142
Epoch 4/1000
1974/1974 [==============================] - 0s 186us/step - loss: 18.9530
Epoch 5/1000
1974/1974 [==============================] - 0s 188us/step - loss: 18.1876
Epoch 6/1000
1974/1974 [==============================] - 0s 177us/step - loss: 19.1091
Epoch 7/1000
1974/1974 [==============================] - 0s 189us/step - loss: 18.1484
Epoch 8/1000
1974/1974 [==============================] - 0s 174us/step - loss: 17.6654
Epoch 9/1000
1974/1974 [==============================] - 0s 220us/step - loss: 17.0712
Epoch 10/1000
1974/1974 [==============================] - 0s 219us/step - loss: 16.6407
Epoch 11/1000
1974/1974 [==============================] - 0s 230us/step - loss: 18.7434
Epoch 12/1000
1974/1974 [=====

1974/1974 [==============================] - 1s 367us/step - loss: 7.4193
Epoch 186/1000
1974/1974 [==============================] - 1s 335us/step - loss: 6.9231
Epoch 187/1000
1974/1974 [==============================] - 1s 358us/step - loss: 7.5439
Epoch 188/1000
1974/1974 [==============================] - 1s 263us/step - loss: 6.7570
Epoch 189/1000
1974/1974 [==============================] - 1s 290us/step - loss: 7.0997
Epoch 190/1000
1974/1974 [==============================] - 0s 237us/step - loss: 6.7201
Epoch 191/1000
1974/1974 [==============================] - 1s 263us/step - loss: 7.2373
Epoch 192/1000
1974/1974 [==============================] - 1s 345us/step - loss: 6.9204
Epoch 193/1000
1974/1974 [==============================] - 1s 319us/step - loss: 7.1762
Epoch 194/1000
1974/1974 [==============================] - 1s 283us/step - loss: 6.9334
Epoch 195/1000
1974/1974 [==============================] - 1s 390us/step - loss: 7.0364
Epoch 196/1000
1974/1974 [==========

1974/1974 [==============================] - 0s 168us/step - loss: 7.2340
Epoch 370/1000
1974/1974 [==============================] - 1s 295us/step - loss: 7.3636
Epoch 371/1000
1974/1974 [==============================] - 0s 242us/step - loss: 6.8402
Epoch 372/1000
1974/1974 [==============================] - 1s 278us/step - loss: 6.7442
Epoch 373/1000
1974/1974 [==============================] - 1s 267us/step - loss: 7.4577
Epoch 374/1000
1974/1974 [==============================] - 0s 252us/step - loss: 6.8274
Epoch 375/1000
1974/1974 [==============================] - 0s 244us/step - loss: 6.5487
Epoch 376/1000
1974/1974 [==============================] - 1s 329us/step - loss: 6.7437
Epoch 377/1000
1974/1974 [==============================] - 1s 526us/step - loss: 6.9414
Epoch 378/1000
1974/1974 [==============================] - 1s 504us/step - loss: 7.3749
Epoch 379/1000
1974/1974 [==============================] - 1s 406us/step - loss: 7.0642
Epoch 380/1000
1974/1974 [==========

1974/1974 [==============================] - 1s 278us/step - loss: 6.8823
Epoch 554/1000
1974/1974 [==============================] - 1s 287us/step - loss: 7.3776
Epoch 555/1000
1974/1974 [==============================] - 1s 309us/step - loss: 6.7474
Epoch 556/1000
1974/1974 [==============================] - 1s 265us/step - loss: 7.0285
Epoch 557/1000
1974/1974 [==============================] - 0s 243us/step - loss: 7.1032
Epoch 558/1000
1974/1974 [==============================] - 1s 285us/step - loss: 6.5297
Epoch 559/1000
1974/1974 [==============================] - 1s 269us/step - loss: 7.4187
Epoch 560/1000
1974/1974 [==============================] - 1s 309us/step - loss: 6.6162
Epoch 561/1000
1974/1974 [==============================] - 0s 246us/step - loss: 6.8670
Epoch 562/1000
1974/1974 [==============================] - 1s 281us/step - loss: 6.5994
Epoch 563/1000
1974/1974 [==============================] - 1s 268us/step - loss: 6.9529
Epoch 564/1000
1974/1974 [==========

1974/1974 [==============================] - 1s 291us/step - loss: 7.2638
Epoch 738/1000
1974/1974 [==============================] - 0s 252us/step - loss: 7.1174
Epoch 739/1000
1974/1974 [==============================] - 1s 262us/step - loss: 6.6542
Epoch 740/1000
1974/1974 [==============================] - 1s 287us/step - loss: 6.6792
Epoch 741/1000
1974/1974 [==============================] - 0s 253us/step - loss: 6.8110
Epoch 742/1000
1974/1974 [==============================] - 1s 264us/step - loss: 6.9384
Epoch 743/1000
1974/1974 [==============================] - 1s 311us/step - loss: 6.5039
Epoch 744/1000
1974/1974 [==============================] - 1s 284us/step - loss: 7.0405
Epoch 745/1000
1974/1974 [==============================] - 0s 245us/step - loss: 6.4850
Epoch 746/1000
1974/1974 [==============================] - 1s 386us/step - loss: 6.3771
Epoch 747/1000
1974/1974 [==============================] - 1s 347us/step - loss: 6.7733
Epoch 748/1000
1974/1974 [==========

1974/1974 [==============================] - 1s 268us/step - loss: 6.6639
Epoch 830/1000
1974/1974 [==============================] - 0s 217us/step - loss: 6.4279
Epoch 831/1000
1974/1974 [==============================] - 0s 209us/step - loss: 6.7049
Epoch 832/1000
1974/1974 [==============================] - 1s 256us/step - loss: 6.9291
Epoch 833/1000
1974/1974 [==============================] - 0s 229us/step - loss: 6.8222
Epoch 834/1000
1974/1974 [==============================] - 1s 284us/step - loss: 6.6560
Epoch 835/1000
1974/1974 [==============================] - 1s 326us/step - loss: 6.6167
Epoch 836/1000
1974/1974 [==============================] - 1s 361us/step - loss: 7.0797
Epoch 837/1000
1974/1974 [==============================] - 1s 343us/step - loss: 6.9813
Epoch 838/1000
1974/1974 [==============================] - 1s 279us/step - loss: 6.5207
Epoch 839/1000
1974/1974 [==============================] - 1s 371us/step - loss: 6.8722
Epoch 840/1000
1974/1974 [==========

In [120]:
model.evaluate(x_test, y_test)

744/744 [==============================] - 0s 130us/step


38.387668907001455